# Hyper parameters optimization #

## Parameters cell ##

Parameters are overiddent by papermill when run inside DVC stages



In [1]:
number_of_folds = 5 # this sets global setting of which how many bootstraps to use
repeats = 10
n_trials = 50
#first round of optimization
lgb_params = {"bagging_fraction": 0.9522534844058304, 
              "boosting_type": "dart", 
              "objective": "regression",
              "feature_fraction": 0.42236910941558053, 
              "lambda_l1": 0.020847266580277746, 
              "lambda_l2": 2.8448564854773326, 
              "learning_rate": 0.11484015430016059, 
              "max_depth": 3, 
              "max_leaves": 35, 
              "min_data_in_leaf": 9,
              "num_iterations": 150
             }
debug_local = True #to use local version

In [2]:
from pathlib import Path
import sys
import inspect

local = (Path("..") / "yspecies").resolve()
if debug_local and local.exists():
  sys.path.insert(0, Path("..").as_posix())
  #sys.path.insert(0, local.as_posix())
  print("extending pathes with local yspecies")
  print(sys.path)
  %load_ext autoreload
  %autoreload 2

extending pathes with local yspecies
['..', '/data/sources/yspecies/notebooks', '/opt/miniconda3/envs/yspecies/lib/python38.zip', '/opt/miniconda3/envs/yspecies/lib/python3.8', '/opt/miniconda3/envs/yspecies/lib/python3.8/lib-dynload', '', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages/IPython/extensions', '/home/antonkulaga/.ipython']


In [3]:
from dataclasses import dataclass, replace
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from typing import *
from yspecies.dataset import *
from yspecies.utils import *
from yspecies.workflow import TupleWith, Repeat, Collect
from yspecies.config import *
from yspecies.preprocess import FeatureSelection, DataExtractor
from yspecies.partition import DataPartitioner, PartitionParameters
from yspecies.selection import ShapSelector
from yspecies.tuning import Tune
from yspecies.models import ResultsCV, CrossValidator
import optuna
from optuna import Study, Trial

In [5]:
#settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import pprint
pp = pprint.PrettyPrinter(indent=4)

#charts settings
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
%matplotlib auto
plt.ioff()
set_matplotlib_formats('svg')

Using matplotlib backend: TkAgg


### Loading data ###
Let's load data from species/genes/expressions selected by select_samples.py notebook

In [6]:
from pathlib import Path
locations: Locations = Locations("./") if Path("./data").exists() else Locations("../")

In [7]:
data = ExpressionDataset.from_folder(locations.interim.selected)
data

expressions,genes,species,samples,Genes Metadata,Species Metadata
"(445, 12340)",12340,39,445,"(12340, 2)","(40, 19)"


## Checking that crossvalidation works ##

Deciding on selection parameters (which fields to include, exclude, predict)

In [8]:
partition_params = PartitionParameters(number_of_folds, 1, 2, [],  42)


In [9]:
partition_cv_pipe = Pipeline([
    ('partitioner', DataPartitioner()),
    ('prepare_for_partitioning', TupleWith(lgb_params)),
    ('crossvalidator', CrossValidator())
]
)

In [10]:
repeated_cv_pipe =  Repeat(partition_cv_pipe, repeats, lambda x,i: (x[0], replace(x[1], seed = i)))
cv_pipeline =  Pipeline([
    ('extractor', DataExtractor()),
    ('prepare_for_partitioning', TupleWith(partition_params)), # to extract the data required for ML from the dataset
    ("partition_cv", repeated_cv_pipe)
    ]
    )

## Setting up features to select ##

In [11]:
selection = select_lifespan = FeatureSelection(
    samples = ["tissue","species"], #samples metadata to include
    species =  [], #species metadata other then Y label to include
    exclude_from_training = ["species"],  #exclude some fields from LightGBM training
    to_predict = "lifespan", #column to predict
    categorical = ["tissue"])

In [12]:
select_lifespan = selection
select_mass = replace(selection, to_predict = "mass_g")
select_gestation = replace(selection, to_predict = "gestation")
select_mtgc = replace(selection, to_predict = "mtgc")

## Checking cross-validation ##

In [13]:
cv_res = cv_pipeline.fit_transform((data, select_lifespan))
ResultsCV.take_best(cv_res)

Found `num_iterations` in params. Will use it instead of argument
Early stopping is not available in dart mode


[150]	cv_agg's l1: 7.0071 + 9.23023	cv_agg's l2: 158.687 + 254.039	cv_agg's huber: 5.95545 + 8.27924
[150]	cv_agg's l1: 2.38291 + 0.276588	cv_agg's l2: 33.5298 + 2.66791	cv_agg's huber: 1.80887 + 0.242136
[150]	cv_agg's l1: 2.42461 + 0.104708	cv_agg's l2: 33.4744 + 1.72278	cv_agg's huber: 1.85087 + 0.093391
[150]	cv_agg's l1: 13.0004 + 5.23252	cv_agg's l2: 201.022 + 104.496	cv_agg's huber: 11.307 + 4.68721
[150]	cv_agg's l1: 2.49413 + 0.377353	cv_agg's l2: 34.1965 + 5.6828	cv_agg's huber: 1.91509 + 0.312529
[150]	cv_agg's l1: 2.41961 + 0.2428	cv_agg's l2: 33.7931 + 3.63635	cv_agg's huber: 1.84299 + 0.212897
[150]	cv_agg's l1: 2.34481 + 0.134413	cv_agg's l2: 33.1724 + 2.30235	cv_agg's huber: 1.78264 + 0.113431
[150]	cv_agg's l1: 6.88053 + 9.15438	cv_agg's l2: 154.905 + 245.647	cv_agg's huber: 5.84682 + 8.20896
[150]	cv_agg's l1: 8.46022 + 7.76312	cv_agg's l2: 137.511 + 158.647	cv_agg's huber: 7.25739 + 6.94678
[150]	cv_agg's l1: 2.40884 + 0.102162	cv_agg's l2: 33.255 + 2.66855	cv_agg's 

1.3033874283531373

In [14]:
[c.last("huber")for c in cv_res]

[5.955452624279493,
 1.808867406228326,
 1.850873638610524,
 11.307032326232683,
 1.9150938190603142,
 1.8429891370236628,
 1.7826372580360528,
 5.846817085256929,
 7.257394629787859,
 1.8364748053060722]

# Optimization #

In [15]:
url = f'sqlite:///' +str((locations.metrics.lifespan / "study.sqlite").absolute())
print('loading (if exists) study from '+url)
storage = optuna.storages.RDBStorage(
    url=url
    #engine_kwargs={'check_same_thread': False}
)

loading (if exists) study from sqlite:////data/sources/yspecies/notebooks/../data/metrics/lifespan/study.sqlite


In [16]:
def objective_parameters(trial: Trial) -> dict:
    return {
        'objective': 'regression',
        'metric': {'mae', 'mse', 'huber'},
        'verbosity': -1,
        'boosting_type': trial.suggest_categorical('boosting_type', ['dart', 'gbdt']),
        'lambda_l1': trial.suggest_uniform('lambda_l1', 0.01, 4.0),
        'lambda_l2': trial.suggest_uniform('lambda_l2', 0.01, 4.0),
        'max_leaves': trial.suggest_int("max_leaves", 15, 25),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.3, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.3, 1.0),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.1),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 3, 8),
        'drop_rate': trial.suggest_uniform('drop_rate', 0.1, 0.3),
        "verbose": -1
    }
optimization_parameters = objective_parameters

In [17]:
from yspecies.workflow import SplitReduce

def side(i: int):
    print(i)
    return i

prepare_partition = SplitReduce(
    outputs = DataPartitioner(), 
    split = lambda x: [(x[0], replace(partition_params, seed=side(x[2])))], 
    reduce = lambda x, output: (output[0], x[1]) 
)                               
partition_and_cv = Pipeline(
    [
        ("prepare partition", prepare_partition),
        ('crossvalidator', CrossValidator())
    ]
)

partition_and_cv_repeat =  Pipeline([
    ("repeat_cv_pipe", Repeat(partition_and_cv, repeats, 
                              lambda x, i: [x[0], x[1], i] )),
    ("collect_mean", Collect(fold=lambda results: np.array([r.last("huber") for r in results]).mean()))
    ]
    )

p = Pipeline([
     ('extractor', DataExtractor()),
     ('tune', Tune(partition_and_cv_repeat, n_trials = n_trials, parameters_space = optimization_parameters))    
])

In [ ]:
best = p.fit_transform((data, select_lifespan))
best

0
Training until validation scores don't improve for 10 rounds
[200]	cv_agg's l1: 0.0450354 + 0.00954981	cv_agg's l2: 0.0103292 + 0.00492932	cv_agg's huber: 0.00516459 + 0.00246466
Did not meet early stopping. Best iteration is:
[200]	cv_agg's l1: 0.0450354 + 0.00954981	cv_agg's l2: 0.0103292 + 0.00492932	cv_agg's huber: 0.00516459 + 0.00246466
1
Training until validation scores don't improve for 10 rounds
[200]	cv_agg's l1: 0.0470572 + 0.0170571	cv_agg's l2: 0.0526272 + 0.0440876	cv_agg's huber: 0.0166453 + 0.0105423
Did not meet early stopping. Best iteration is:
[200]	cv_agg's l1: 0.0470572 + 0.0170571	cv_agg's l2: 0.0526272 + 0.0440876	cv_agg's huber: 0.0166453 + 0.0105423
2
Training until validation scores don't improve for 10 rounds
[200]	cv_agg's l1: 0.0439069 + 0.00955033	cv_agg's l2: 0.0134161 + 0.00863501	cv_agg's huber: 0.00666935 + 0.00424484
Did not meet early stopping. Best iteration is:
[200]	cv_agg's l1: 0.0439069 + 0.00955033	cv_agg's l2: 0.0134161 + 0.00863501	cv_agg'

[I 2020-08-13 13:11:57,854] Trial 0 finished with value: 0.010980089050296113 and parameters: {'boosting_type': 'gbdt', 'lambda_l1': 1.0015394482144673, 'lambda_l2': 0.2835427804155781, 'max_leaves': 25, 'max_depth': 8, 'feature_fraction': 0.5260057910641318, 'bagging_fraction': 0.8034577607874316, 'learning_rate': 0.08361855604732259, 'min_data_in_leaf': 8, 'drop_rate': 0.17304435068507087}. Best is trial 0 with value: 0.010980089050296113.


0
Training until validation scores don't improve for 10 rounds
[200]	cv_agg's l1: 0.0776314 + 0.0084358	cv_agg's l2: 0.0939079 + 0.0543168	cv_agg's huber: 0.0333004 + 0.0112798
Did not meet early stopping. Best iteration is:
[200]	cv_agg's l1: 0.0776314 + 0.0084358	cv_agg's l2: 0.0939079 + 0.0543168	cv_agg's huber: 0.0333004 + 0.0112798
1
Training until validation scores don't improve for 10 rounds
[200]	cv_agg's l1: 0.110016 + 0.017932	cv_agg's l2: 0.278031 + 0.103563	cv_agg's huber: 0.0551277 + 0.00990949
Did not meet early stopping. Best iteration is:
[200]	cv_agg's l1: 0.110016 + 0.017932	cv_agg's l2: 0.278031 + 0.103563	cv_agg's huber: 0.0551277 + 0.00990949
2
Training until validation scores don't improve for 10 rounds
[200]	cv_agg's l1: 0.138959 + 0.0322669	cv_agg's l2: 0.112423 + 0.0667931	cv_agg's huber: 0.045048 + 0.0203393
Did not meet early stopping. Best iteration is:
[200]	cv_agg's l1: 0.138959 + 0.0322669	cv_agg's l2: 0.112423 + 0.0667931	cv_agg's huber: 0.045048 + 0.020

[I 2020-08-13 13:16:58,428] Trial 1 finished with value: 0.04559555824096727 and parameters: {'boosting_type': 'gbdt', 'lambda_l1': 2.676957087704102, 'lambda_l2': 3.6037561309244555, 'max_leaves': 22, 'max_depth': 4, 'feature_fraction': 0.8901988789100828, 'bagging_fraction': 0.419316655841148, 'learning_rate': 0.0871130581891701, 'min_data_in_leaf': 8, 'drop_rate': 0.2974903100721361}. Best is trial 0 with value: 0.010980089050296113.


0
[200]	cv_agg's l1: 6.24192 + 0.580073	cv_agg's l2: 157.981 + 10.7662	cv_agg's huber: 5.22768 + 0.522413
1
[200]	cv_agg's l1: 5.75528 + 2.2462	cv_agg's l2: 86.5246 + 68.0048	cv_agg's huber: 4.79413 + 2.02863
2
[200]	cv_agg's l1: 6.20322 + 0.901862	cv_agg's l2: 160.325 + 22.3078	cv_agg's huber: 5.19431 + 0.814497
3
[200]	cv_agg's l1: 5.8487 + 1.9852	cv_agg's l2: 86.8602 + 63.1184	cv_agg's huber: 4.8774 + 1.78974
4
[200]	cv_agg's l1: 6.52095 + 0.888011	cv_agg's l2: 166.926 + 26.9495	cv_agg's huber: 5.48079 + 0.798582
5
[200]	cv_agg's l1: 6.2494 + 0.529756	cv_agg's l2: 158.797 + 14.8541	cv_agg's huber: 5.23699 + 0.474595
6
[200]	cv_agg's l1: 6.41949 + 0.894288	cv_agg's l2: 162.183 + 24.7007	cv_agg's huber: 5.38403 + 0.8021
7
[200]	cv_agg's l1: 6.41689 + 1.03865	cv_agg's l2: 162.082 + 30.0381	cv_agg's huber: 5.38928 + 0.940932
8
[200]	cv_agg's l1: 6.37457 + 1.18028	cv_agg's l2: 160.364 + 26.5352	cv_agg's huber: 5.35188 + 1.06598
9
[200]	cv_agg's l1: 5.85683 + 1.9423	cv_agg's l2: 87.8842 +

...

0
Training until validation scores don't improve for 10 rounds
[200]	cv_agg's l1: 3.34867 + 0.197085	cv_agg's l2: 36.4929 + 1.60302	cv_agg's huber: 2.6409 + 0.182949
Did not meet early stopping. Best iteration is:
[200]	cv_agg's l1: 3.34867 + 0.197085	cv_agg's l2: 36.4929 + 1.60302	cv_agg's huber: 2.6409 + 0.182949
1
Training until validation scores don't improve for 10 rounds
[200]	cv_agg's l1: 3.33949 + 0.162398	cv_agg's l2: 36.6233 + 1.52747	cv_agg's huber: 2.62166 + 0.145462
Did not meet early stopping. Best iteration is:
[200]	cv_agg's l1: 3.33949 + 0.162398	cv_agg's l2: 36.6233 + 1.52747	cv_agg's huber: 2.62166 + 0.145462
2
Training until validation scores don't improve for 10 rounds
[200]	cv_agg's l1: 3.41226 + 0.38266	cv_agg's l2: 37.255 + 4.87362	cv_agg's huber: 2.68539 + 0.351772
Did not meet early stopping. Best iteration is:
[200]	cv_agg's l1: 3.41226 + 0.38266	cv_agg's l2: 37.255 + 4.87362	cv_agg's huber: 2.68539 + 0.351772
3
Training until validation scores don't improve 

[I 2020-08-13 16:01:24,060] Trial 30 finished with value: 2.5575267265649337 and parameters: {'boosting_type': 'gbdt', 'lambda_l1': 0.3129688096801219, 'lambda_l2': 1.991185777439485, 'max_leaves': 25, 'max_depth': 6, 'feature_fraction': 0.6372396313609705, 'bagging_fraction': 0.9954450788353645, 'learning_rate': 0.010676455186618111, 'min_data_in_leaf': 3, 'drop_rate': 0.1128903622879699}. Best is trial 21 with value: 0.0008705968725506693.


0
Training until validation scores don't improve for 10 rounds
[200]	cv_agg's l1: 0.0143323 + 0.00450174	cv_agg's l2: 0.00705737 + 0.00328396	cv_agg's huber: 0.00352868 + 0.00164198
Did not meet early stopping. Best iteration is:
[200]	cv_agg's l1: 0.0143323 + 0.00450174	cv_agg's l2: 0.00705737 + 0.00328396	cv_agg's huber: 0.00352868 + 0.00164198
1
Training until validation scores don't improve for 10 rounds
[200]	cv_agg's l1: 0.0064663 + 0.00367633	cv_agg's l2: 0.00143742 + 0.00216499	cv_agg's huber: 0.000718712 + 0.0010825
Did not meet early stopping. Best iteration is:
[200]	cv_agg's l1: 0.0064663 + 0.00367633	cv_agg's l2: 0.00143742 + 0.00216499	cv_agg's huber: 0.000718712 + 0.0010825
2
Training until validation scores don't improve for 10 rounds
[200]	cv_agg's l1: 0.00492762 + 0.00254996	cv_agg's l2: 0.000738274 + 0.000717882	cv_agg's huber: 0.000369137 + 0.000358941
Did not meet early stopping. Best iteration is:
[200]	cv_agg's l1: 0.00492762 + 0.00254996	cv_agg's l2: 0.000738274

[I 2020-08-13 16:37:13,331] Trial 31 finished with value: 0.0008990867149605796 and parameters: {'boosting_type': 'gbdt', 'lambda_l1': 0.03681782972214824, 'lambda_l2': 2.949544867837493, 'max_leaves': 24, 'max_depth': 5, 'feature_fraction': 0.8089662722148161, 'bagging_fraction': 0.9405487078401557, 'learning_rate': 0.0919560265454591, 'min_data_in_leaf': 4, 'drop_rate': 0.11534494712759874}. Best is trial 21 with value: 0.0008705968725506693.


0
Training until validation scores don't improve for 10 rounds
[200]	cv_agg's l1: 0.0125466 + 0.00112387	cv_agg's l2: 0.00243554 + 0.00154275	cv_agg's huber: 0.00121777 + 0.000771374
Did not meet early stopping. Best iteration is:
[200]	cv_agg's l1: 0.0125466 + 0.00112387	cv_agg's l2: 0.00243554 + 0.00154275	cv_agg's huber: 0.00121777 + 0.000771374
1
Training until validation scores don't improve for 10 rounds
[200]	cv_agg's l1: 0.0102017 + 0.00218254	cv_agg's l2: 0.00192589 + 0.00137061	cv_agg's huber: 0.000962945 + 0.000685305
Did not meet early stopping. Best iteration is:
[200]	cv_agg's l1: 0.0102017 + 0.00218254	cv_agg's l2: 0.00192589 + 0.00137061	cv_agg's huber: 0.000962945 + 0.000685305
2
Training until validation scores don't improve for 10 rounds
[200]	cv_agg's l1: 0.00865016 + 0.00186888	cv_agg's l2: 0.000993717 + 0.000277164	cv_agg's huber: 0.000496859 + 0.000138582
Did not meet early stopping. Best iteration is:
[200]	cv_agg's l1: 0.00865016 + 0.00186888	cv_agg's l2: 0.000

In [ ]:
best["metric"] =  ["mae", "mse", "huber"]
best['objective'] = 'regression'
best

## Getting shap results with the best parameters ##

In [ ]:
def make_shap(params: dict):
    partition_shap_pipe = Pipeline([
        ("partitioner", DataPartitioner()),
        ('prepare_for_partitioning', TupleWith(lgb_params)),
        ("shap_computation", ShapSelector())
    ]
    )
    repeated_cv =  Repeat(partition_shap_pipe, repeats, lambda x,i: (x[0], replace(x[1], seed = i)))
    return Pipeline([
        ('extractor', DataExtractor()),
        ('prepare_for_partitioning', TupleWith(params)), # to extract the data required for ML from the dataset
        ("partition_shap", repeated_cv)]
        )